In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
mol_str = """
0 1
o
h 1 1.0
h 1 1.0 2 104.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)

cavity_options = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'cis',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'davidson_roots' : 20,
    'nact_orbs' : 0,
    'nact_els' : 0
}

psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [3]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 0

print(omega_val)

# lambda vector
lambda_vector = np.array([0., 0., 0.0])
#lambda_vector = np.array([0.5, 0.5, 0.5])

n_act_orb=5
#number of active electrons
n_act_el=4
# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)


0

Start SCF iterations:

Canonical RHF One-electron energy = -121.6753455400170196
CQED-RHF One-electron energy      = -121.6753455400170196
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -74.9648590960667320   dE = -7.49649E+01   dRMS = 8.59412E-15
SCF Iteration   2: Energy = -74.9648590960666610   dE =  7.10543E-14   dRMS = 2.85301E-15
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -74.96485910 hartree
Psi4  SCF energy: -74.96485910 hartree
printing CIS tuples
[(0, 1, 2, 3, 4), (0, 1, 2, 3, 5), (0, 1, 2, 3, 6), (0, 1, 2, 4, 5), (0, 1, 2, 4, 6), (0, 1, 3, 4, 5), (0, 1, 3, 4, 6), (0, 2, 3, 4, 5), (0, 2, 3, 4, 6), (1, 2, 3, 4, 5), (1, 2, 3, 4, 6)]
subspace size is too large, try smaller size


SystemExit: 

/opt/homebrew/Caskroom/miniforge/base/envs/psi4/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

# davidson eigenvalues and eigenvectors are stored in attributes
# H2_PF.cis_e and H2_PF.cis_c for now!


In [ ]:
print("checking to see if the Davidson Roots Agree!")
print(e_fci[:14])
print(H2_PF.cis_e[:14] )
print(np.allclose(e_fci[:14] , H2_PF.cis_e[:14]))


In [ ]:

H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




In [ ]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)